In [ ]:
#libraries
import numpy as np
import imageio.v2 as io
import os
from tomopy.prep.normalize import normalize as flat_dark
from tomopy.recon.algorithm import recon
from tomopy.recon.rotation import write_center as c_o_r

#### Step 1 - Nomalise the projections with darks and flats

In [ ]:
# load projection data from tiffs in folder
path = 'D:/21_ILL_SP_before_15mm_15s/Proj'
proj_arr = []
for i in os.listdir(path):
    proj = io.imread(path+'/'+i)
    proj_arr.append(proj)
proj_arr = np.array(proj_arr)

amiin = np.sqrt(np.amin(proj_arr)**2)
mean = np.mean((proj_arr+amiin))
std = np.std(proj_arr+amiin)
maax = mean + 5*std
miin = mean - 5*std
proj_arr_8 = []
for k in range(proj.shape[0]): #reslice so that vertically sliced 
    im = ((vol[k]+amiin)/(maax)) #convert to 8bit
    im = np.where(im < (miin/maax), 0, im)
    im = np.where(im > 1, 1, im)*255
    im = im - ((mean/maax)*225 - 127.5)
    proj_arr_8.append((im).astype(np.uint8))
proj_arr_8 = np.array(proj_arr_8)

io.volwrite('8bit_proj.tiff', proj_arr_8, format='tiff', bigtiff=True)

In [ ]:
# load flat and dark
path_f = 'D:/20_flats_15mmPinhole_15s'
flat_arr = []
for i in os.listdir(path_f):
    f = io.imread(path_f+'/'+i)
    flat_arr.append(f)
flat_arr = np.array(flat_arr)
#for k in range(flat_arr.shape[0]):
#    im = im + flat_arr[k]
#flat = im / flat_arr.shape[0]


path_d = 'D:/22_darks_15mm_15s'
dark_arr = []
for l in os.listdir(path_d):
    d = io.imread(path_d+'/'+l)
    dark_arr.append(d)
dark_arr = np.array(dark_arr)
#for m in range(dark_arr.shape[0]):
#    im = im + dark_arr[k]
#dark = im / dark_arr.shape[0]

In [ ]:
# apply normalisation and save to disk
proj_arr = flat_dark(proj_arr, flat_arr, dark_arr, averaging='mean')
io.volwrite('normed_proj.tiff', proj_arr, format='tiff', bigtiff=True)

### Step 2 - Define array of acquisition angles for each projection 

In [ ]:
angle = 0 
angles = []
for j in range(1216):
    angles.append(angle)
    angle += (360/1215)
angles = np.array(angles)
angles = angles * (np.pi/180)

### Step 3 - Find the Centre of Roatation for the reconstruction 

In [ ]:
c_o_r(proj_arr, angles, cen_range = [(int(proj_arr.shape[0]/2)-50), (int(proj_arr.shape[0]/2)+50), 5], algorithm = 'fbp', filter_name = 'ramlak')
# Then load data into image viwer and see which centre of rotation is the best and choose best value
centre = 750

### Step 4 - Reconstruct the projections

In [ ]:
recon_vol = recon(proj_arr, angles, center = centre, algorithm = 'fbp', filter_name = 'ramlak')
io.volwrite('recon.tiff', recon_vol, format='tiff', bigtiff=True)